In [28]:
%%writefile reduce.py
from guang.Utils.toolsFunc import probar
import os
def reduce_from_duration(filelist):
    """
    return: reduce_list, reduced_list, IDX
    """
    reduce_list = []
    reduced_list = []
    IDX = []
    for idx, i in probar(filelist):
        try:
            print(i)
            if librosa.get_duration(filename = i) < 10:
                reduce_list.append(i)
            else:
                reduced_list.append(i)
                IDX.append(idx)
        except:
            print(f'file {i} open failed, and has been ignored')
            continue
    print(F'The number that need to be deleted:{len(reduce_list)}')
    return reduce_list, reduced_list, IDX

if __name__ == "__main__":
    from glob import glob
    flist = glob(r"data/*")
#     print(flist)
    reduce_from_duration(flist)

Overwriting reduce.py


In [30]:
!python reduce.py

__version__: 0.0.7.5.8

0.00% 	  0.0|infs 	data\000001.wav
file data\000001.wav open failed, and has been ignored

3.45% 	0'0.0"|0'0.0"	Expect:12-28 23:25:30 	data\000002.wav
file data\000002.wav open failed, and has been ignored

6.90% 	0'0.0"|0'0.0"	Expect:12-28 23:25:30 	data\000003.wav
file data\000003.wav open failed, and has been ignored

10.34% 	0'0.0"|0'0.0"	Expect:12-28 23:25:30 	data\000004.wav
file data\000004.wav open failed, and has been ignored

13.79% 	0'0.0"|0'0.0"	Expect:12-28 23:25:30 	data\000005.wav
file data\000005.wav open failed, and has been ignored

17.24% 	0'0.0"|0'0.0"	Expect:12-28 23:25:30 	data\000006.wav
file data\000006.wav open failed, and has been ignored

20.69% 	0'0.0"|0'0.0"	Expect:12-28 23:25:30 	data\000007.wav
file data\000007.wav open failed, and has been ignored

24.14% 	0'0.0"|0'0.0"	Expect:12-28 23:25:30 	data\000008.wav
file data\000008.wav open failed, and has been ignored

27.59% 	0'0.0"|0'0.0"	Expect:12-28 23:25:30 	data\000009.wav
file da